#06 토픽 모델링(Topic Modeling)-20뉴스그룹

머신러닝 기반의 토픽 모델은 숨겨진 주제를 효과적으로 표현할 수 있는 중심 단어를 함축적으로 추출함.

**LDA 토픽 모델링**

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#모토사이클,야구,그래픽스,윈도우즈, 중동, 기독교, 전자공학, 의학 8개 주제를 추출
cats =['rec.motorcycles','rec.sport.baseball','comp.graphics','comp.windows.x','talk.politics.mideast','soc.religion.christian','sci.electronics','sci.med']

#위에서 cats 변수로 기재된 카테고리만 추출.fetch_20newsgroups()의 categories에 cats 입력
news_df = fetch_20newsgroups(subset='all',remove=('headers','footers','quotes'),categories=cats,random_state=0)

#LDA는 Count기반의 벡터화만 적용합니다.
count_vect = CountVectorizer(max_df=0.95,max_features=1000,min_df=2,stop_words='english',ngram_range=(1,2))

feat_vect = count_vect.fit_transform(news_df.data)
print('CountVectorizer Shape:',feat_vect.shape)

CountVectorizer Shape: (7862, 1000)


토큰의 빈도가 max_df로 지정한 값을 초과 하거나 min_df로 지정한 값보다 작은 경우에는 무시한다.  
max_features : 추출할 단어 피처의 개수

CountVectorizer 객체 변수인 feat_vect 모두 7862개의 문서가 1000개의 피처로 구성된 행렬 데이터

In [4]:
#n_components : 토픽 개수
lda = LatentDirichletAllocation(n_components=8,random_state=0)
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=8, random_state=0)

In [5]:
print(lda.components_.shape)
#components_ : 개별 토픽별로 각 word 피처가 얼마나 많이 그 토픽에 할당됐는지에 대한 수치를 가진다.
#높은 값일수록 해당 word 피처는 그 토픽의 중심 word
lda.components_

(8, 1000)


array([[3.60992018e+01, 1.35626798e+02, 2.15751867e+01, ...,
        3.02911688e+01, 8.66830093e+01, 6.79285199e+01],
       [1.25199920e-01, 1.44401815e+01, 1.25045596e-01, ...,
        1.81506995e+02, 1.25097844e-01, 9.39593286e+01],
       [3.34762663e+02, 1.25176265e-01, 1.46743299e+02, ...,
        1.25105772e-01, 3.63689741e+01, 1.25025218e-01],
       ...,
       [3.60204965e+01, 2.08640688e+01, 4.29606813e+00, ...,
        1.45056650e+01, 8.33854413e+00, 1.55690009e+01],
       [1.25128711e-01, 1.25247756e-01, 1.25005143e-01, ...,
        9.17278769e+01, 1.25177668e-01, 3.74575887e+01],
       [5.49258690e+01, 4.47009532e+00, 9.88524814e+00, ...,
        4.87048440e+01, 1.25034678e-01, 1.25074632e-01]])

components_ : array[8,4000]  
8개의 토픽별로 1000개의 word 피처가 해당 토픽별로 연관도 값을 가지고 있다.  
components_array의 0번째row, 10번째 col에 있는 값   
= Topic #0에 대해서 피처 벡터화된 행렬에서 10번째 칼럼에 해당하는 피처가 Topic #0에 연관되는 수치 값

display_topics() : 각 토픽별로 연관도가 높은 순으로 word를 나열해보자.

In [12]:
def display_topics(model,feature_names, no_top_words):
    for topic_index,topic in enumerate(model.components_):
        print('Topic #',topic_index)

        #components_array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array 인덱스를 반환.
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes = topic_word_indexes[:no_top_words]

        #top_indexes 대상인 인덱스별로 feature_names에 해당하는 word featrue 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])
        print(feature_concat)


In [9]:
#CountVectorizer객체 내의 전체 word의 명칭을 get_features_names()를 통해 추출
feature_names = count_vect.get_feature_names()
len(feature_names)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


1000

In [13]:
#토픽별 가장 연관도가 높은 word를 15개만 추출
display_topics(lda,feature_names,15)

Topic # 0
year 10 game medical health team 12 20 disease cancer 1993 games years patients good
Topic # 1
don just like know people said think time ve didn right going say ll way
Topic # 2
image file jpeg program gif images output format files color entry 00 use bit 03
Topic # 3
like know don think use does just good time book read information people used post
Topic # 4
armenian israel armenians jews turkish people israeli jewish government war dos dos turkey arab armenia 000
Topic # 5
edu com available graphics ftp data pub motif mail widget software mit information version sun
Topic # 6
god people jesus church believe christ does christian say think christians bible faith sin life
Topic # 7
use dos thanks windows using window does display help like problem server need know run


#07 문서 군집화 소개와 실습(Opinion Review 데이터 세트)

**문서 군집화**   
: 비슷한 텍스트 구성의 문서를 군집화(Clustering) 하는 것  
학습 데이터 세트가 필요 없는 비지도학습 기반으로 동작

##Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기

전체 51개의 파일이 토요타와 같은 자동차 브랜드에 대한 평가와 아이팟 나노(ipod nano)의 음질과 같은 다양한 전자 제품과 호텔 서비스 등에 대한 리뷰 내용

In [36]:
#여러 개의 파일을 DataFrame으로 로딩해보자.
import pandas as pd
import glob, os

path = '/content/drive/MyDrive/ESAA(22-1)/Week10/Opinosis Opinion Review Data set/OpinosisDataset1.0/topics'
#path로 지정한 디렉터리 밑에 있는 모든 .data 파일의 파일명을 리스트로 취합.
all_files = glob.glob(os.path.join(path,"*.data"))
filename_list=[]
opinion_text = []

#개별 파일의 파일명은 filename_list로 취합,
#개별 파일의 파일 내용은 DataFrame 로딩 후 sting으로 변환해 opinion_text_list로 취합
for file_ in all_files:
    #개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_,index_col=None,header=0,encoding='latin1')

    #절대 경로로 주어진 확장명을 가공.
    #맨 마지막 .data 확장자도 제거
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]

    #파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가.
    filename_list.append(filename)
    opinion_text.append(df.to_string().replace("    ","")) #첫 공백 제거


In [37]:
document_df = pd.DataFrame({'filename':filename_list,'opinion_text':opinion_text})
document_df.iloc[4,1]

"headphone jack i got a clear case for it and it  i got a clear case for it and it like prvents me from being able to put the jack all the way in so the sound can b messsed up or i can get it in there and its playing well them go to move or something and it slides out .\n0 Picture and sound quality are excellent for this typ of devic .\n1  Great sound, easy to load & use once I read the instructions .\n2 The music I loaded sounds clear and crisp .\n3As for the overall sound I would give it a 8 out of 10, and the volume offers itself loud enough to tune out everything else at your local gym .\n4sound and picture quality is great !\n5  I notice that the audio playback has improved alot over the previous generations I could hear hissing in the third generation and didn't really remember how if sounded on the fourth generation because I didn't have it along time .\n6  Sound quality on the built in speaker is not great but through headphones is awesome !\n7  Video camera quality and sound a

In [38]:
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb ...
1,directions_garmin_nuvi_255W_gps,You also get upscale features like spoken di...
2,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh ..."
3,display_garmin_nuvi_255W_gps,3 quot widescreen display was a bonus .\n0 ...
4,sound_ipod_nano_8gb,headphone jack i got a clear case for it and i...


In [23]:
ex_file_path = '/content/drive/MyDrive/ESAA(22-1)/Week10/Opinosis Opinion Review Data set/OpinosisDataset1.0/topics/accuracy_garmin_nuvi_255W_gps.txt.data'
file_path_1 = ex_file_path.split('/')[-1]
file_path_1

'accuracy_garmin_nuvi_255W_gps.txt.data'

In [25]:
file_path_2 = file_path_1.split('.')[0]
file_path_2

'accuracy_garmin_nuvi_255W_gps'

문서를 TF-IDF 형태로 피처 벡터화해보자.  
tokenizer는 이전 예제에서 Lemmatization을 구현한 LemNormalize() 함수를 이용할 것

In [39]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

#단어 원형 추출 함수
lemmar = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

#특수 문자 사전 생성: {33: None ...}
#ord() : 아스키 코드 생성
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation)

#LemNormalize() 함수
def LemNormalize(text):
    #텍스트 소문자 변경 후 특수 문자 제거
    text_new = text.lower().translate(remove_punct_dict)

    #단어 토큰화
    word_tokens = nltk.word_tokenize(text_new)

    #단어 원형 추출
    return LemTokens(word_tokens)



*   string.punctuation : 느낌표, 물음표, 더하기 등의 문자들
*   dict((ord(punct),None) for punct in string.punctuation) : 해당 문자 사전을 생성
*   text.lower().translate(remove_punct_dict) 로 문자 사전에 따라 None 으로 변환한다.
*   단어 토큰화 후에 토큰 별로 원형을 추출한다.





In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1,2),min_df=0.05,max_df=0.85)

#opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

LookupError: ignored

In [41]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

#################
## 벡터화
#################
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english' , \
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

LookupError: ignored

#08 문서 유사도

##문서 유사도 측정 방법 - 코사인 유사도

문서와 문서 간의 유사도 비교는 일반적으로 코사인 유사도(Cosine Similarity)를 사용한다.   
코사인 유사도는 벡터와 벡터 간의 유사도를 비교할 때 벡터의 크기보다는 벡터의 상호 방향성이 얼마나 유사한지에 기반한다.즉, 코사인 유사도는 두 벡터 사이의 사잇각을 구해서 얼마나 유사한지 수치로 적용한 것

##두 벡터 사잇각

In [50]:
#cos_similarity() 함수 
#두 개의 넘파이 배열에 대한 코사인 유사도를 구한다.
import numpy as np

def cos_similarity(v1,v2):
    dot_product = np.dot(v1,v2)
    l2_norm = (np.sqrt(sum(np.square(v1)))*np.sqrt(sum(np.square(v2))))
    similarity = dot_product/l2_norm
    
    return similarity

doc_list로 정의된 3개의 간단한 문서의 유사도를 비교하기 위해 이 문서를 TF-IDF로 벡터화된 행렬로 변환한다.

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

doc_list = ['if you take the blue pill, the story ends',
            'if you take the red pill, you stay in Wonderland',
            'if you take the red pill, I show you how deep the rabbit hole goes']

tfidf_vect_simple = TfidfVectorizer()
feature_vect_simple = tfidf_vect_simple.fit_transform(doc_list)
print(feature_vect_simple.shape)

(3, 18)


반환된 행렬은 희소 행렬이므로 앞에서 작성한 cos_similarity() 함수의 인자인 array로 만들기 위해 밀집 행렬로 변환한 뒤 다시 각각을 배열로 변환한다.

In [45]:
#TfidfVectorizer로 transform() 한 결과는 희소 행렬이므로 밀집 행렬로 변환
feature_vect_dense = feature_vect_simple.todense()
feature_vect_dense

matrix([[0.4155636 , 0.        , 0.4155636 , 0.        , 0.        ,
         0.        , 0.24543856, 0.        , 0.24543856, 0.        ,
         0.        , 0.        , 0.        , 0.4155636 , 0.24543856,
         0.49087711, 0.        , 0.24543856],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.23402865, 0.39624495, 0.23402865, 0.        ,
         0.3013545 , 0.        , 0.39624495, 0.        , 0.23402865,
         0.23402865, 0.39624495, 0.4680573 ],
        [0.        , 0.30985601, 0.        , 0.30985601, 0.30985601,
         0.30985601, 0.18300595, 0.        , 0.18300595, 0.30985601,
         0.23565348, 0.30985601, 0.        , 0.        , 0.18300595,
         0.3660119 , 0.        , 0.3660119 ]])

In [48]:
#첫 번째 문장과 두 번째 문장의 피처 벡터 추출
vect1 = np.array(feature_vect_dense[0]).reshape(-1, )
vect2 = np.array(feature_vect_dense[1]).reshape(-1, )
vect1

array([0.4155636 , 0.        , 0.4155636 , 0.        , 0.        ,
       0.        , 0.24543856, 0.        , 0.24543856, 0.        ,
       0.        , 0.        , 0.        , 0.4155636 , 0.24543856,
       0.49087711, 0.        , 0.24543856])

In [51]:
#첫 번째 문장과 두 번째 문장의 피처 벡터로 두 개 문장의 코사인 유사도 추출
similarity_simple = cos_similarity(vect1,vect2)
print('문장1,문장2 Cosine 유사도:{0:.3f}'.format(similarity_simple))

문장1,문장2 Cosine 유사도:0.402


In [52]:
#첫 번째 문장과 세 번째 문장의 피처 벡터로 두 개 문장의 코사인 유사도 추출
vect1 = np.array(feature_vect_dense[0]).reshape(-1, )
vect3 = np.array(feature_vect_dense[2]).reshape(-1, )
similarity_simple = cos_similarity(vect1,vect3)
print('문장1,문장3 Cosine 유사도:{0:.3f}'.format(similarity_simple))

문장1,문장3 Cosine 유사도:0.404


In [54]:
#두 번째 문장과 세 번째 문장의 피처 벡터로 두 개 문장의 코사인 유사도 추출
vect2 = np.array(feature_vect_dense[1]).reshape(-1, )
vect3 = np.array(feature_vect_dense[2]).reshape(-1, )
similarity_simple = cos_similarity(vect2,vect3)
print('문장2,문장3 Cosine 유사도:{0:.3f}'.format(similarity_simple))

문장2,문장3 Cosine 유사도:0.456


사이킷런은 코사인 유사도를 측정하기 위해 sklearn.metrics.pairwise.cosine_similarity API 제공한다.  
첫 번째 파라미터 : 비교 기준이 되는 문서의 피처 행렬  
두 번째 파라미터 : 비교되는 문서의 피처 행렬  
cosine_similarity()는 희소 행렬, 밀집 행렬 모두가 가능하며, 행렬 또는 배열 모두 가능하다.

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(feature_vect_simple[0],feature_vect_simple)
print(similarity_simple_pair)

[[1.         0.40207758 0.40425045]]


In [56]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(feature_vect_simple[0],feature_vect_simple)
print(similarity_simple_pair)

[[0.40207758 0.40425045]]


In [57]:
similarity_simple_pair = cosine_similarity(feature_vect_simple,feature_vect_simple)
print(similarity_simple_pair)
print('shape:',similarity_simple_pair.shape)


[[1.         0.40207758 0.40425045]
 [0.40207758 1.         0.45647296]
 [0.40425045 0.45647296 1.        ]]
shape: (3, 3)


#09 한글 텍스트 처리 - 네이버 영화 평점 감성 분석

In [58]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/ESAA(22-1)/Week10/네이버 영화 평점/ratings_train.txt',sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [59]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [61]:
train_df.isnull().sum()

id          0
document    5
label       0
dtype: int64

0과 1의 비율이 어느 한쪽으로 치우치지 않고 균등한 분포를 나타내고 있다.


*   train_df의 경우 리뷰 텍스트를 가지는 'document'칼럼에 Null이 일부 존재하므로 이 값은 공백으로 변환한다. 
*   문자가 아닌 숫자의 경우 단어적인 의미로 부족하므로 파이썬의 정규 표현식 모듈인 re를 이용해 이 역시 공백으로 변환한다.



In [63]:
import re

train_df = train_df.fillna(' ')
#정규 표현식을 이용해 숫자를 공백으로 변경(정규 표현식으로 \d는 숫자를 의미함.)
train_df['document'] = train_df['document'].apply(lambda x :re.sub(r"\d+"," ",x))

In [64]:
train_df.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [65]:
#테스트 데이터 세트를 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('/content/drive/MyDrive/ESAA(22-1)/Week10/네이버 영화 평점/ratings_test.txt',sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x :re.sub(r"\d+"," ",x))

#id 칼럼 삭제 수행
train_df.drop('id',axis=1,inplace=True)
test_df.drop('id',axis=1,inplace=True)

In [66]:
test_df.isnull().sum()

document    0
label       0
dtype: int64



1.   각 문장 한글 형태소 분석을 통해 형태소 단어로 토큰화(SNS 분석에 적합한 Twitter 클래스를 이용)
2.   TF-IDF 방식으로 단어 벡터화



In [ ]:
install.packages('k')

In [68]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 27.7 MB/s 


In [69]:
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(text):
    #입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 변환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

#Twitter 객체의 morphs() 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2)
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer,ngram_range=(1,2),min_df=3,max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [73]:
#로지스틱 회귀를 이용해 감성 분석 분류 수행
lg_clf = LogisticRegression(random_state=0)

#파라미터 C 최적화를 위해 GridSearchCV를 이용
params = {'C': [1,3.5,4.5,5.5,10]}
grid_cv = GridSearchCV(lg_clf,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv.fit(tfidf_matrix_train,train_df['label'])
print(grid_cv.best_params_,round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


{'C': 3.5} 0.8593


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [75]:
from sklearn.metrics import accuracy_score

#학습 데이터를 적용한 TfidfVectorizer를 이용해 테스트 데이터를 TF-IDF 값으로 피처 변환함.
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

#classifier는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도:',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도: 0.86186
